[View in Colaboratory](https://colab.research.google.com/github/restrepo/WOSplus/blob/master/test_sample.ipynb)

In [1]:
%%bash
# Check if colaboratory was launched and install missing requirements
if [ "$(pwd)" == /content ];then
    pip install openpyxl xlrd unidecode python-levenshtein requests_testadapter > /dev/null
    git clone https://github.com/restrepo/WOSplus.git > /dev/null
    mv WOSplus/wosplus/* .
fi

Cloning into 'WOSplus'...


In [0]:
import wosplus

In [0]:
WOS_file='CIB_Wos.xlsx'
SCI_file='CIB_Scielo.xlsx'
SCP_file='CIB_Scopus.csv'

cib=wosplus.wosplus('drive.cfg')

cib.load_biblio(WOS_file)
cib.load_biblio(SCI_file,prefix='SCI')
cib.load_biblio(SCP_file,prefix='SCP')

print('before merge: {}'.format( cib.WOS.shape[0]+cib.SCI.shape[0]+cib.SCP.shape[0] )  )

cib.merge(left='WOS',right='SCI')

if True:
    print('intial: {}'.format( cib.WOS.shape[0]+cib.SCI.shape[0]) )
    print('final : {}'.format(  cib.WOS_SCI.shape) )

cib.merge(left='WOS_SCI',right='SCP')

if True:
    print('intial: {}'.format( cib.WOS_SCI.shape[0]+cib.SCP.shape[0]) )
    print('final : {}'.format(  cib.WOS_SCI_SCP.shape) )

/usr/local/lib/python3.5/dist-packages/wosplus/_wos_scp.py:54: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  return df.rename_axis( dict( (key,prefix+'_'+key) for key in df.columns.values) , axis=1)


before merge: 856
....intial: 488
final : (464, 96)
............intial: 832
final : (574, 141)
before merge: 856
....intial: 488
final : (464, 96)
............intial: 832
final : (574, 141)
